In [ ]:
!pip install pytigergraph

In [ ]:
import pandas as pd
import pyTigerGraph as tg 

server = 'https://medialab2.i.tgcloud.io/' 
username = 'tigergraph'
password = 'tigergraph' 
apiToken = 'mnte371fka2ug1simvi6ehi8gsq69fav'

conn = tg.TigerGraphConnection(host=server, graphname='medialab', username=username, password=password, apiToken = apiToken)

In [ ]:
print(conn.gsql('ls', options=[]))

---- Graph medialab
Vertex Types:
- VERTEX drug(PRIMARY_ID id STRING, canonicalSmiles STRING, inchiKey STRING, drugType STRING, blackBoxWarning STRING, name STRING, yearOfFirstApproval STRING, maximumClinicalTrialPhase STRING, parentId STRING, hasBeenWithdrawn STRING, isApproved STRING, withdrawnNotice STRING, tradeNames STRING, synonyms STRING, crossReferences STRING, childChemblIds STRING, linkedTargets STRING, linkedDiseases STRING, description STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX _target(PRIMARY_ID id STRING, approvedSymbol STRING, biotype STRING, transcriptIds STRING, genomicLocation STRING, alternativeGenes STRING, approvedName STRING, go STRING, hallmarks STRING, synonyms STRING, symbolSynonyms STRING, nameSynonyms STRING, functionDescriptions STRING, subcellularLocations STRING, targetClass STRING, obsoleteSymbols STRING, obsoleteNames STRING, _constraint STRING, tep STRING, proteinIds STRING, dbXrefs STRING, chemicalProbes STRING,

# Create graph schema

In [ ]:
# change to True to delele graph data (for subsequent re-creation of Graph schema)

if False:
    print(conn.gsql('''
    USE GRAPH medialab
    DROP QUERY helloworld
    DROP GRAPH medialab
    DROP EDGE disease_target, drug_target, drug_disease
    DROP VERTEX drug,_target,disease
    DROP JOB tmp_job1
    '''))

In [ ]:
print(conn.gsql('''
CREATE GRAPH medialab()

CREATE SCHEMA_CHANGE JOB medialab_schema_job FOR GRAPH medialab{
ADD VERTEX drug (
    PRIMARY_ID id                 STRING,
    canonicalSmiles               STRING,
    inchiKey                      STRING,
    drugType                      STRING,
    blackBoxWarning               STRING,
    name                          STRING,
    yearOfFirstApproval           STRING,
    maximumClinicalTrialPhase     STRING,
    parentId                      STRING,
    hasBeenWithdrawn              STRING,
    isApproved                    STRING,
    withdrawnNotice               STRING,
    tradeNames                    STRING,
    synonyms                      STRING,
    crossReferences               STRING,
    childChemblIds                STRING,
    linkedTargets                 STRING,
    linkedDiseases                STRING,
    description                   STRING) WITH primary_id_as_attribute="true";

ADD VERTEX _target(
    PRIMARY_ID id           STRING,
    approvedSymbol          STRING,
    biotype                 STRING,
    transcriptIds           STRING,
    genomicLocation         STRING,
    alternativeGenes        STRING,
    approvedName            STRING,
    go                      STRING,
    hallmarks               STRING,
    synonyms                STRING,
    symbolSynonyms          STRING,
    nameSynonyms            STRING,
    functionDescriptions    STRING,
    subcellularLocations    STRING,
    targetClass             STRING,
    obsoleteSymbols         STRING,
    obsoleteNames           STRING,
    _constraint             STRING,
    tep                     STRING,
    proteinIds              STRING,
    dbXrefs                 STRING,
    chemicalProbes          STRING,
    homologues              STRING,
    tractability            STRING,
    safetyLiabilities       STRING,
    pathways                STRING) WITH primary_id_as_attribute="true";
    
    
ADD VERTEX disease(
    PRIMARY_ID id          STRING,
    code                   STRING,
    dbXRefs                STRING,
    description            STRING,
    name                   STRING,
    directLocationIds      STRING,
    obsoleteTerms          STRING,
    parents                STRING,
    sko                    STRING,
    synonyms               STRING,
    ancestors              STRING,
    descendants            STRING,
    children               STRING,
    therapeuticAreas       STRING,
    indirectLocationIds    STRING,
    ontology               STRING) WITH primary_id_as_attribute="true";

ADD UNDIRECTED EDGE disease_target (FROM disease, TO _target, score DOUBLE, evidenceCount INT);
ADD UNDIRECTED EDGE drug_disease (FROM drug, TO disease, 
    indications STRING, 
    approvedIndications STRING,
    indicationCount INT,
    maxPhaseForIndication INT);  //IF 4, mean approved drug for the disease
    
ADD UNDIRECTED EDGE drug_target (FROM drug, TO _target,
	actionType STRING,
	mechanismOfAction STRING, 
	//chemblIds STRING,
	targetName STRING,
	targetType STRING,
	//targets STRING,
	references STRING);

}
RUN SCHEMA_CHANGE JOB medialab_schema_job
DROP JOB medialab_schema_job
'''))

Stopping GPE GSE RESTPP
Successfully stopped GPE GSE RESTPP in 22.225 seconds
Starting GPE GSE RESTPP
Successfully started GPE GSE RESTPP in 0.171 seconds
The graph medialab is created.
Successfully created schema change jobs: [medialab_schema_job].

Current graph version 0
Trying to add vertex drug.
Trying to add vertex _target.
Trying to add vertex disease.
Trying to add edge disease_target.
Trying to add edge drug_disease.
Trying to add edge drug_target.
Kick off job medialab_schema_job

Graph medialab update to new version 1
The job medialab_schema_job completes in 13.123 seconds!
Successfully dropped jobs on the graph 'medialab': [medialab_schema_job].


# Create and install GSQL query

In [ ]:
print(conn.gsql('''
USE GRAPH medialab
DROP QUERY by_disease_find_novel_drugs_from_drugs_of_similar_diseases
CREATE QUERY by_disease_find_novel_drugs_from_drugs_of_similar_diseases(vertex<disease> d, float similarity) FOR GRAPH medialab SYNTAX v2 { 
  /*****
  * d input: 
  *     follicular lymphoma (disease: MONDO_0018906)
  *     COVID-19 (disease: MONDO_0100096)
  *     prostate cancer (disease:MONDO_0008315)
  * similarity input:
  *     1.0
  * result: 
  *      found: drug (id: CHEMBL1201560, approvedPEGINTERFERON ALFA-2A

  ******/
  OrAccum<BOOL> @visited, @associatedDrugs;
  SumAccum<double> @scoreByD, @lengthASqr, @lengthBSqr, @dotProductAB, @cosineSimilarity;
  ListAccum<EDGE> @@edgeList;
  
  seed = {d};
  
  targetsScoredByDisease = SELECT t FROM seed:s-(disease_target:e)-_target:t 
            ACCUM t.@visited = true, t.@scoreByD = e.score;
  
  diseasesScoredSameTarget = SELECT tgt FROM targetsScoredByDisease:s-(disease_target:e)-disease:tgt
    WHERE tgt !=  d
    ACCUM tgt.@lengthASqr += s.@scoreByD * s.@scoreByD,
          tgt.@lengthBSqr += e.score * e.score,
          tgt.@dotProductAB += s.@scoreByD * e.score
    POST-ACCUM
          tgt.@cosineSimilarity =
            tgt.@dotProductAB / (sqrt(tgt.@lengthASqr) * sqrt(tgt.@lengthBSqr))
    HAVING tgt.@cosineSimilarity >= similarity
    ORDER BY tgt.@cosineSimilarity DESC;
    //LIMIT k1;
    
  approved_drugs = SELECT t FROM seed:s -(drug_disease:e)-:t 
        ACCUM t.@associatedDrugs = true;
  
  novel_drugs = SELECT t FROM diseasesScoredSameTarget:s-(drug_disease:e)-drug:t 
        WHERE t.@associatedDrugs == false;
  
  prioritized_drugs = SELECT s FROM novel_drugs:s-(drug_target:e1)-_target:t1-(disease_target:e2)-disease:t2  
        WHERE t2 == d and e2.score > 0.5 
        ACCUM @@edgeList += e1, @@edgeList += e2; 
        //ORDER BY e2.score DESC;
  
  PRINT @@edgeList;
  /* PRINT targetsScoredByDisease;
  PRINT diseasesScoredSameTarget;
  PRINT novel_drugs;
  PRINT prioritized_drugs;
  */
}
INSTALL QUERY by_disease_find_novel_drugs_from_drugs_of_similar_diseases
'''))

Using graph 'medialab'
Semantic Check Fails: These queries could not be found anywhere: [by_disease_find_novel_drugs_from_drugs_of_similar_diseases].
Successfully created queries: [by_disease_find_novel_drugs_from_drugs_of_similar_diseases].
Start installing queries, about 1 minute ...
by_disease_find_novel_drugs_from_drugs_of_similar_diseases query: curl -X GET 'https://127.0.0.1:9000/query/medialab/by_disease_find_novel_drugs_from_drugs_of_similar_diseases?d=VALUE&similarity=VALUE'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
Select 'm1' as compile server, now connecting ...
Node 'm1' is prepared as compile server.

Query installation finished.


# Query tigergraph database

In [ ]:
df = conn.runInstalledQuery('by_disease_find_novel_drugs_from_drugs_of_similar_diseases', {'d':'MONDO_0008315', 'similarity': 1.0})